<a href="https://colab.research.google.com/github/Rakib-hosen/word_embedding/blob/master/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2020-08-17 17:36:47--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 173.194.69.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  26.9MB/s    in 0.2s    

2020-08-17 17:36:49 (26.9 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [2]:
import csv
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#hyperparameters

vocab_size = 1000
embedding_dim = 16
oov_token = '<OOV>'
trunc_type = 'post'
train_portion = 0.8
padding_type = 'post'
max_len = 120


In [4]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
with open('/tmp/bbc-text.csv', 'r') as csvfile:
  reader = csv.reader(csvfile,delimiter=',')
  next(reader)
  for label, sentence in reader:
    labels.append(label)
    for word in stopwords:
      token = ' '+ word + ' '
      sentence = sentence.replace(token, ' ')
      sentence = sentence.replace('  ', ' ')
    sentences.append(sentence)


print(len(labels))
print(labels[0])
print(len(sentences))
print(sentences[0])


2225
tech
2225
tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv will radically different five years time. according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend programmes content will delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices. one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes like us s tivo uk s sky+ system allow people record store play pause forward wind tv programmes want. essentially technology allows much personalised tv. also built-in high-definition tv sets big business japan us slower take off europe lack high-definition programming. not can people forward wind adverts can also forget abiding network channel schedules putting together a-la-carte entertainment. us network

In [5]:
train_size = int(len(sentences)*train_portion)
train_sentences = sentences[:train_size]
train_labels = labels[:train_size]
test_sentences = sentences[train_size:]
test_labels = labels[train_size:]
print(len(train_sentences),len(test_sentences))

1780 445


In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
train_sequences_padded = pad_sequences(sequences,padding=padding_type, maxlen=max_len, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_sequences_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_len, truncating=trunc_type)



In [11]:
l_tokenizer = Tokenizer()
l_tokenizer.fit_on_texts(labels)

train_labels_sequences = np.array(l_tokenizer.texts_to_sequences(train_labels))
test_labels_sequences = np.array(l_tokenizer.texts_to_sequences(test_labels))

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 150       
Total params: 16,558
Trainable params: 16,558
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_sequences_padded,train_labels_sequences,
          validation_data=(test_sequences_padded,test_labels_sequences),
          epochs=20,
          verbose= 2)

Epoch 1/20
56/56 - 0s - loss: 0.9698 - accuracy: 0.7753 - val_loss: 0.9183 - val_accuracy: 0.7775
Epoch 2/20
56/56 - 0s - loss: 0.7914 - accuracy: 0.8506 - val_loss: 0.7588 - val_accuracy: 0.8652
Epoch 3/20
56/56 - 0s - loss: 0.6473 - accuracy: 0.9022 - val_loss: 0.6443 - val_accuracy: 0.8876
Epoch 4/20
56/56 - 0s - loss: 0.5342 - accuracy: 0.9258 - val_loss: 0.5510 - val_accuracy: 0.8989
Epoch 5/20
56/56 - 0s - loss: 0.4457 - accuracy: 0.9399 - val_loss: 0.4780 - val_accuracy: 0.9079
Epoch 6/20
56/56 - 0s - loss: 0.3762 - accuracy: 0.9466 - val_loss: 0.4297 - val_accuracy: 0.9056
Epoch 7/20
56/56 - 0s - loss: 0.3194 - accuracy: 0.9522 - val_loss: 0.3820 - val_accuracy: 0.9124
Epoch 8/20
56/56 - 0s - loss: 0.2743 - accuracy: 0.9590 - val_loss: 0.3484 - val_accuracy: 0.9146
Epoch 9/20
56/56 - 0s - loss: 0.2376 - accuracy: 0.9624 - val_loss: 0.3233 - val_accuracy: 0.9236
Epoch 10/20
56/56 - 0s - loss: 0.2080 - accuracy: 0.9669 - val_loss: 0.2996 - val_accuracy: 0.9236
Epoch 11/20
56/56 -